In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
%%capture
%pip install -U transformers accelerate
# %pip install pyaudio wave torchaudio datasets


In [3]:
# from transformers import pipeline

# pipe = pipeline("text-to-speech", model="suno/bark-small", device=0)  # Use GPU (device 0)


In [4]:
# text = "\n\nI cannot predict the type of cancer based on a single sentence. Can I help you with anything else"
# output = pipe(text)
# from IPython.display import Audio
# Audio(output["audio"], rate=output["sampling_rate"])

In [5]:
import pandas as pd
df = pd.read_csv('/kaggle/input/biomedical-text-publication-classification/alldata_1_for_kaggle.csv',encoding="latin1")
df.drop(columns='Unnamed: 0', inplace=True)
df.columns = ['class', 'doc']
df = df.reindex(['doc', 'class'], axis=1)
df

,doc,class
0,Thyroid surgery in children in a single insti...,Thyroid_Cancer
1,""" The adopted strategy was the same as that us...",Thyroid_Cancer
2,coronary arterybypass grafting thrombosis ï¬b...,Thyroid_Cancer
3,Solitary plasmacytoma SP of the skull is an u...,Thyroid_Cancer
4,This study aimed to investigate serum matrix ...,Thyroid_Cancer
...,...,...
7565,we report the case of a 24yearold man who pres...,Colon_Cancer
7566,among synchronous colorectal cancers scrcs rep...,Colon_Cancer
7567,the heterogeneity of cancer cells is generally...,Colon_Cancer
7568,"""adipogenesis is the process through which mes...",Colon_Cancer


In [6]:
df[:10]

,doc,class
0,Thyroid surgery in children in a single insti...,Thyroid_Cancer
1,""" The adopted strategy was the same as that us...",Thyroid_Cancer
2,coronary arterybypass grafting thrombosis ï¬b...,Thyroid_Cancer
3,Solitary plasmacytoma SP of the skull is an u...,Thyroid_Cancer
4,This study aimed to investigate serum matrix ...,Thyroid_Cancer
5,This study was performed to explore the effec...,Thyroid_Cancer
6,This study was performed assess the clinical ...,Thyroid_Cancer
7,Journal of International Medical Research  Th...,Thyroid_Cancer
8,Gastric cancer GC persists as a worldwide pub...,Thyroid_Cancer
9,Scars Burns HealingVolume  reuse guideli...,Thyroid_Cancer


In [7]:
# Shuffle the dataset for randomness
dataset = df.sample(frac=1, random_state=65).reset_index(drop=True)

In [8]:
data = dataset[:20]
data.head(20)

,doc,class
0,"""This phenomenon which we did not observe in o...",Lung_Cancer
1,"""The protocol was approved by the Institutiona...",Lung_Cancer
2,""" it is estimated that around  of patients w...",Colon_Cancer
3,"""cellular recognition of microbial dna is an e...",Colon_Cancer
4,"""Ann Surg233: 18919411176124 29 YamashitaJI K...",Lung_Cancer
5,99mtclabeled nanocolloid drugs development m...,Thyroid_Cancer
6,in a novel coronavirus sarscov2 was found to ...,Colon_Cancer
7,"""Statistical analysis We used Chi-square test ...",Lung_Cancer
8,"""These results have also been included in our ...",Lung_Cancer
9,"""Therefore gathering a complete profile of mut...",Lung_Cancer


In [9]:
data['class'].unique()

array(['Lung_Cancer', 'Colon_Cancer', 'Thyroid_Cancer'], dtype=object)

In [10]:
# data.iloc[[1], [0]]

In [11]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline, TextStreamer)

import torch

In [12]:
base_model = "/kaggle/input/llama-3.2/transformers/1b-instruct/1"

tokenizer = AutoTokenizer.from_pretrained(base_model)

In [13]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    return_dict=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

In [14]:
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
if model.config.pad_token_id is None:
    model.config.pad_token_id = model.config.eos_token_id

In [15]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

Device set to use cuda:0


In [16]:
import json
# Prepare input for the model
results = []
for _, row in data.iterrows():
    doc = row["doc"]
    messages = [
        {"role": "system", "content": """You are an AI model trained to predict cancer types like 'Lung_Cancer', 'Colon_Cancer' and  'Thyroid_Cancer'  based on medical texts.your 
        answer must be one words answer like ('Lung_Cancer', 'Colon_Cancer', 'Thyroid_Cancer').if you have conflit in 
        the understanding then only return the empty """},
        {"role": "user", "content": f"Now predict the type of cancer for the following text:\nText: {doc}\nClass:"}
    ]
    # Generate prompt
    prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    
    # Generate output
    outputs = pipe(prompt, max_new_tokens=20, do_sample=True)
    
    # Extract the generated text
    generated_text = outputs[0]["generated_text"].split(
                "<|start_header_id|>assistant<|end_header_id|>\n\n"
            )[1]
    # Append to results
    results.append({"input": doc, "predicted_class": generated_text})

# Save results in JSON format
output_path = "cancer_predictions_1b_zero_shot.json"
with open(output_path, "w") as f:
    json.dump(results, f, indent=4)

print(f"Predictions saved to {output_path}")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Predictions saved to cancer_predictions_1b_zero_shot.json


In [17]:
example_text = "\n".join([f"Text: {row['doc']}\nClass: {row['class']}" for _, row in dataset[20:25].iterrows()])

In [18]:
# Prepare input for the model
results = []
for _, row in data.iterrows():
    doc = row["doc"]
    
    # Create the prompt with few-shot examples
    messages = [
        {"role": "system", "content": """You are an AI model trained to predict cancer types like 'Lung_Cancer', 'Colon_Cancer' and  'Thyroid_Cancer'  based on medical texts.your 
        answer must be one words answer like ('Lung_Cancer', 'Colon_Cancer', 'Thyroid_Cancer').if you have conflit in 
        the understanding then only return the empty """},
        {"role": "user", "content": f"Here are some examples:\n{example_text}\n\nNow predict the type of cancer for the following text:\nText: {doc}\nClass:"}
    ]
    
    # Generate prompt
    prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    
    # Generate output
    outputs = pipe(prompt, max_new_tokens=120, do_sample=True)
    
    # Extract the generated text (predicted class)
    generated_text = outputs[0]["generated_text"].split(
                "<|start_header_id|>assistant<|end_header_id|>\n\n"
            )[1]
   
    # Append to results
    results.append({"input": doc, "predicted_class": generated_text})

# Save results in JSON format
output_path = "cancer_predictions_1b_few_shot.json"
with open(output_path, "w") as f:
    json.dump(results, f, indent=4)

print(f"Predictions saved to {output_path}")

Predictions saved to cancer_predictions_1b_few_shot.json
